In [21]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage.morphology import binary_dilation, binary_fill_holes
# needs to have been preprocessed

Drive = "Z"
Subfolder = "ProcessedData" 
animal = "Quille"
date = "2023-09-28"
preprocess = "PreprocessedFiles\\"
process_directory = f"{Drive}:\\{Subfolder}\\{animal}\\{date}\\suite2p\\{preprocess}"
suite2p_directory = f"{Drive}:\\{Subfolder}\\{animal}\\{date}\\suite2p\\"


In [28]:
# functions needed

# originally from suite2p!
def boundary(ypix, xpix):
    """ returns pixels of mask that are on the exterior of the mask """
    ypix = np.expand_dims(ypix.flatten(), axis=1)
    xpix = np.expand_dims(xpix.flatten(), axis=1)
    npix = ypix.shape[0]
    if npix > 0:
        msk = np.zeros((np.ptp(ypix) + 6, np.ptp(xpix) + 6), "bool")
        msk[ypix - ypix.min() + 3, xpix - xpix.min() + 3] = True
        msk = binary_dilation(msk)
        msk = binary_fill_holes(msk)
        k = np.ones((3, 3), dtype=int)  # for 4-connected
        k = np.zeros((3, 3), dtype=int)
        k[1] = 1
        k[:, 1] = 1  # for 8-connected
        out = binary_dilation(msk == 0, k) & msk

        yext, xext = np.nonzero(out)
        yext, xext = yext + ypix.min() - 3, xext + xpix.min() - 3
    else:
        yext = np.zeros((0,))
        xext = np.zeros((0,))
    return yext, xext
            
def plot_masks(upper_th = 0.6, lower_th = 0.4):
    """
    Plots the outer borders of cells and colours them 
    depending on the probability that it is a inhibitory neuron.
    """
    mask_red = np.zeros(red_img.shape)
    mask_blue = np.zeros(red_img.shape)
    mask_green = np.zeros(red_img.shape)

    for n in range(stat.shape[0]):
        if red_prob_all[curr_plane][n] > upper_th:
            ypix = stat[n]['ypix'][~stat[n]['overlap']]
            xpix = stat[n]['xpix'][~stat[n]['overlap']]
            out_y, out_x = boundary(ypix, xpix)  
            mask_red[out_y, out_x] = 1
        elif red_prob_all[curr_plane][n] < upper_th and red_prob_all[curr_plane][n] > lower_th:
            ypix = stat[n]['ypix'][~stat[n]['overlap']]
            xpix = stat[n]['xpix'][~stat[n]['overlap']]
            out_y, out_x = boundary(ypix, xpix)  
            mask_blue[out_y, out_x] = 1
        elif red_prob_all[curr_plane][n] < lower_th:
            ypix = stat[n]['ypix'][~stat[n]['overlap']]
            xpix = stat[n]['xpix'][~stat[n]['overlap']]
            out_y, out_x = boundary(ypix, xpix)  
            mask_green[out_y, out_x] = 1
    fig, ax = plt.subplots(1, 2, figsize = (12,6))
    ax[0].imshow(mean_img_green, cmap="gist_gray")
    ax[1].imshow(red_img, cmap="gist_gray")

    colored_img_red = np.zeros(red_img.shape + (4,))
    colored_img_blue = np.zeros(red_img.shape + (4,))
    colored_img_green = np.zeros(red_img.shape + (4,))

    colored_img_red[mask_red == 1, 0] = 1
    colored_img_red[mask_red == 1, 1] = 0
    colored_img_red[mask_red == 1, 2] = 0
    colored_img_red[mask_red == 1, 3] = 1

    colored_img_blue[mask_blue == 1, 0] = 0
    colored_img_blue[mask_blue == 1, 1] = 0
    colored_img_blue[mask_blue == 1, 2] = 1
    colored_img_blue[mask_blue == 1, 3] = 1

    colored_img_green[mask_green == 1, 0] = 0
    colored_img_green[mask_green == 1, 1] = 1
    colored_img_green[mask_green == 1, 2] = 0
    colored_img_green[mask_green == 1, 3] = 1

    ax[0].imshow(colored_img_red)
    ax[0].imshow(colored_img_blue)
    ax[0].imshow(colored_img_green)
    ax[0].set_title("chan1")

    ax[1].imshow(colored_img_red)
    ax[1].imshow(colored_img_blue)
    ax[1].imshow(colored_img_green)
    ax[1].set_title("chan2")

    ax[1].text(450, 10, "Vgat+ > " + str(upper_th), c="red", weight="bold")
    ax[1].text(450, 30, "unsure", c="blue", weight="bold")
    ax[1].text(450, 50, "Vgat- < " + str(lower_th), c="green", weight="bold")

    plt.show()

    manager = plt.get_current_fig_manager()
    manager.full_screen_toggle()

        

In [22]:
# Define the number of planes you want to process
num_planes = 3  # You can change this to the number of planes you have


red_all = np.array([])
# probability of red
red_prob_all = np.array([])

for plane_num in range(1, num_planes + 1):
    plane_folder = f"{Drive}:\\{Subfolder}\\{animal}\\{date}\\suite2p\\plane{plane_num}"
    red_plane = np.load(os.path.join(plane_folder, "redcell.npy"), allow_pickle=True)
    iscell = np.load(os.path.join(plane_folder, "iscell.npy"), allow_pickle=True)
    iscell = iscell[:, 0].astype(bool)
    red = red_plane[iscell, 0]
    red_prob = red_plane[iscell, 1]
    red_all = np.hstack((red_all, red))
    red_prob_all = np.hstack((red_prob_all, red_prob))
# to determine which cells in gui are the ones we get
cell_ids= np.load(os.path.join(process_directory, "calcium.Ids.npy"), allow_pickle=True)

In [23]:


plane_directories = [suite2p_directory + f"plane{i}" for i in range(1, 2)]

for i, plane_dir in enumerate(plane_directories, start=1):
    planes = np.load(os.path.join(process_directory, "calcium.planes.npy"), allow_pickle=True)
    iscell = np.load(os.path.join(plane_dir, "iscell.npy"), allow_pickle=True)
    iscell = iscell[:, 0].astype(bool)
    ops = np.load(os.path.join(plane_dir, "ops.npy"), allow_pickle=True)
    ops = ops.item()
    curr_plane = np.where(planes == i)[0]

    mean_img_green = ops["meanImg"] * 5
    red_img = ops["meanImg_chan2"]

    stat = np.load(os.path.join(plane_dir, "stat.npy"), allow_pickle=True)
    stat = stat[iscell]




In [29]:
import ipywidgets as widgets
widgets.interact(plot_masks,upper_th = (0,1,0.05), lower_th = (0,1,0.05))

interactive(children=(FloatSlider(value=0.6, description='upper_th', max=1.0, step=0.05), FloatSlider(value=0.…

<function __main__.plot_masks(upper_th=0.6, lower_th=0.4)>

In [36]:
# once you have determined the right threshold, save the data:
upper_th = 0.65
red_certain = np.zeros(red_prob_all.shape[0])
ind_red = np.where(red_prob_all >= upper_th)
red_certain[ind_red] = 1

np.save(os.path.join(process_directory, "red.certain.npy"), red_certain)
